In [1]:
import modes.mode_2 as model2

/Users/bellopasal/opt/anaconda3/envs/currito/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
/Users/bellopasal/PycharmProjects/simulador_1/currito_raspb/bus.jpg

In [ ]:
from picamera2 import Picamera2
import RPi.GPIO as GPIO
import time

# Initialize the Raspberry Pi GPIO
GPIO.setmode(GPIO.BCM)  # Use BCM numbering
SIGNAL_PIN = 17         # GPIO pin to listen for a signal
GPIO.setup(SIGNAL_PIN, GPIO.IN, pull_up_down=GPIO.PUD_DOWN)  # Configure as input with pull-down

# Initialize Picamera2
picam = Picamera2()
picam.configure(picam.create_still_configuration())

def capture_picture():
    """Captures a picture and saves it with a timestamp."""
    timestamp = time.strftime("%Y%m%d_%H%M%S")
    filename = f"image_{timestamp}.jpg"
    picam.start()
    time.sleep(2)  # Allow the camera to adjust (optional, depends on your setup)
    picam.capture_file(filename)
    picam.stop()
    print(f"Captured {filename}")

try:
    print("Waiting for signal...")
    while True:
        # Wait for the signal pin to go HIGH
        if GPIO.input(SIGNAL_PIN) == GPIO.HIGH:
            print("Signal detected!")
            capture_picture()
            time.sleep(1)  # Avoid capturing multiple images for a single signal

except KeyboardInterrupt:
    print("Exiting program...")

finally:
    GPIO.cleanup()  # Clean up GPIO state when exiting



In [2]:
from ultralytics import YOLO

# Load a YOLO11n PyTorch model
model = YOLO("yolo11n.pt")

# Export the model to NCNN format
model.export(format="ncnn")  # creates 'yolo11n_ncnn_model'

# Load the exported NCNN model
ncnn_model = YOLO("yolo11n_ncnn_model")

largest_prediction = None
largest_area = 0
image_path = "images.jpg"

results = ncnn_model(image_path)


largest_prediction = None
largest_area = 0

for result in results:
    # Access detected bounding boxes
    for detection in result.boxes:
        # Print the entire detection object to inspect its contents
        #print(f"Detection: {detection}")
        
        # Check if detection.xywh exists and is in the expected format
        if hasattr(detection, 'xywh'):
            bbox = detection.xywh.tolist() 
            bbox = [item for sublist in bbox for item in sublist] # Convert bbox to a list

            # Ensure bbox has at least 4 elements: [x, y, width, height]
            if len(bbox) >= 4:
                width, height = bbox[2], bbox[3]
                area = width * height

                # If this is the largest box, store its prediction
                if area > largest_area:
                    largest_area = area
                    confidence = detection.conf
                    class_index = detection.cls.item()  # Convert tensor to scalar
                    class_name = result.names[class_index]  # Use scalar index to get the class name
                    
                    largest_prediction = {
                        "class": class_name,
                        "confidence": confidence,
                        "bbox": bbox
                    }
            else:
                print(f"Invalid bbox format (less than 4 elements): {bbox}")
        else:
            print("detection.xywh is missing or not in the expected format.")
    
# Print the largest prediction
if largest_prediction:
    print("Largest prediction:", largest_prediction)
else:
    print("No valid bounding boxes found.")






Ultralytics 8.3.38 🚀 Python-3.10.15 torch-2.2.2 CPU (Apple M1 Pro)
YOLO11n summary (fused): 238 layers, 2,616,248 parameters, 0 gradients, 6.5 GFLOPs


[W NNPACK.cpp:64] Could not initialize NNPACK! Reason: Unsupported hardware.



PyTorch: starting from 'yolo11n.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 84, 8400) (5.4 MB)

TorchScript: starting export with torch 2.2.2...
TorchScript: export success ✅ 2.2s, saved as 'yolo11n.torchscript' (10.5 MB)

NCNN: starting export with NCNN 1.0.20240820...
NCNN: running '/Users/bellopasal/opt/anaconda3/envs/currito/lib/python3.10/site-packages/ultralytics/pnnx yolo11n.torchscript ncnnparam=yolo11n_ncnn_model/model.ncnn.param ncnnbin=yolo11n_ncnn_model/model.ncnn.bin ncnnpy=yolo11n_ncnn_model/model_ncnn.py pnnxparam=yolo11n_ncnn_model/model.pnnx.param pnnxbin=yolo11n_ncnn_model/model.pnnx.bin pnnxpy=yolo11n_ncnn_model/model_pnnx.py pnnxonnx=yolo11n_ncnn_model/model.pnnx.onnx fp16=0 device=cpu inputshape="[1, 3, 640, 640]"'


pnnxparam = yolo11n_ncnn_model/model.pnnx.param
pnnxbin = yolo11n_ncnn_model/model.pnnx.bin
pnnxpy = yolo11n_ncnn_model/model_pnnx.py
pnnxonnx = yolo11n_ncnn_model/model.pnnx.onnx
ncnnparam = yolo11n_ncnn_model/model.ncnn.param
ncnnbin = yolo11n_ncnn_model/model.ncnn.bin
ncnnpy = yolo11n_ncnn_model/model_ncnn.py
fp16 = 0
optlevel = 2
device = cpu
inputshape = [1,3,640,640]f32
inputshape2 = 
customop = 
moduleop = 
############# pass_level0
inline module = torch.nn.modules.linear.Identity
inline module = ultralytics.nn.modules.block.Attention
inline module = ultralytics.nn.modules.block.Bottleneck
inline module = ultralytics.nn.modules.block.C2PSA
inline module = ultralytics.nn.modules.block.C3k
inline module = ultralytics.nn.modules.block.C3k2
inline module = ultralytics.nn.modules.block.DFL
inline module = ultralytics.nn.modules.block.PSABlock
inline module = ultralytics.nn.modules.block.SPPF
inline module = ultralytics.nn.modules.conv.Concat
inline module = ultralytics.nn.modules.con

NCNN: export success ✅ 1.7s, saved as 'yolo11n_ncnn_model' (10.2 MB)

Export complete (4.4s)
Results saved to /Users/bellopasal/PycharmProjects/simulador_1/currito_raspb
Predict:         yolo predict task=detect model=yolo11n_ncnn_model imgsz=640  
Validate:        yolo val task=detect model=yolo11n_ncnn_model imgsz=640 data=/usr/src/ultralytics/ultralytics/cfg/datasets/coco.yaml  
Visualize:       https://netron.app
WARNING ⚠️ Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.
Loading yolo11n_ncnn_model for NCNN inference...

image 1/1 /Users/bellopasal/PycharmProjects/simulador_1/currito_raspb/images.jpg: 640x640 1 orange, 118.0ms
Speed: 3.8ms preprocess, 118.0ms inference, 2.5ms postprocess per image at shape (1, 3, 640, 640)
Largest prediction: {'class': 'orange', 'confidence': tensor([0.4993]), 'bbox': [113.94820404052734, 119.28294372558594, 178.6775360107422, 171.1112

In [3]:
model2.say_something(largest_prediction["class"])

"1. **Oranges are a hybrid:**  The sweet orange we commonly eat isn't a naturally occurring species. It's believed to be a hybrid of the pomelo and mandarin orange, likely originating in Southeast Asia.\n\n2. **Orange peel has surprising uses:**  Beyond just zest for flavoring, orange peel contains essential oils used in perfumes and cleaning products.  It's also rich in pectin, a gelling agent used in jams and jellies, and has been explored for its potential health benefits due to its antioxidant properties.\n"

In [4]:
results

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted p

In [1]:
import modes.mode_2 as mode_2


/Users/bellopasal/opt/anaconda3/envs/currito/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Los últimos presidentes del Gobierno de España han sido:

* **Pedro Sánchez Pérez-Castejón** (desde el 2 de junio de 2018)
* **Mariano Rajoy Brey** (desde el 21 de diciembre de 2011 hasta el 1 de junio de 2018)
* **José Luis Rodríguez Zapatero** (desde el 17 de abril de 2004 hasta el 21 de diciembre de 2011)
* **José María Aznar López** (desde el 5 de mayo de 1996 hasta el 17 de abril de 2004)
* **Felipe González Márquez** (desde el 5 de diciembre de 1982 hasta el 5 de mayo de 1996)


Cabe destacar que antes de la Transición Española, existían Jefes de Estado (como el General Franco) y Presidentes del Gobierno con roles y sistemas diferentes al actual.



In [2]:
o = mode_2.ObjectDetector()

o.detect_objects_in_image(image_path="images.jpeg")


Using cache found in /Users/bellopasal/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-11-27 Python-3.10.15 torch-2.2.2 CPU

Fusing layers... 
[W NNPACK.cpp:64] Could not initialize NNPACK! Reason: Unsupported hardware.
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


array([[[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       ...,

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]]

In [ ]:
from ultralytics import YOLO
import cv2

class RealTimeObjectDetector:
    def __init__(self, model_path, area_threshold):
        """
        Initialize the detector with a YOLO model and area threshold.
        """
        self.model = YOLO(model_path)
        self.area_threshold = area_threshold

    def _process_frame(self, frame):
        """
        Detect objects in a single frame and return the largest prediction
        that exceeds the area threshold.
        """
        results = self.model(frame)
        largest_prediction = None
        largest_area = 0

        for result in results:
            for detection in result.boxes:
                if hasattr(detection, 'xywh'):
                    bbox = detection.xywh.tolist()
                    bbox = [item for sublist in bbox for item in sublist]  # Flatten list

                    if len(bbox) >= 4:
                        width, height = bbox[2], bbox[3]
                        area = width * height

                        if area > self.area_threshold and area > largest_area:
                            largest_area = area
                            confidence = detection.conf
                            class_index = detection.cls.item()  # Convert tensor to scalar
                            class_name = result.names[class_index]  # Map index to class name

                            largest_prediction = {
                                "class": class_name,
                                "confidence": confidence,
                                "bbox": bbox,
                            }
        return largest_prediction

    def start_real_time_detection(self):
        """
        Start real-time object detection from the webcam.
        """
        cap = cv2.VideoCapture(0)  # Use 0 for default webcam; change for other video sources

        if not cap.isOpened():
            print("Error: Unable to access the webcam.")
            return

        while True:
            ret, frame = cap.read()
            if not ret:
                print("Error: Unable to read a frame from the webcam.")
                break

            # Process the current frame
            prediction = self._process_frame(frame)

            # Annotate the frame if a valid prediction exists
            if prediction:
                x, y, w, h = prediction["bbox"]
                # cv2.rectangle(frame, (int(x - w / 2), int(y - h / 2)),
                #               (int(x + w / 2), int(y + h / 2)), (0, 255, 0), 2)
                # cv2.putText(frame, f"{prediction['class']} ({prediction['confidence']:.2f})",
                #             (int(x - w / 2), int(y - h / 2) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

                
                print("Prediction with sufficient area found:", prediction)
                return prediction['class']
                break


            # Exit on 'q' key press
            if (cv2.waitKey(1) & 0xFF == ord('q')) :
                break

        cap.release()
        


real_time_detector = RealTimeObjectDetector("yolo11n_ncnn_model", area_threshold=5000)  # Adjust threshold as needed
real_time_detector.start_real_time_detection()


SyntaxError: invalid syntax (2654069080.py, line 78)